# Taxi and Uber expenses in Brasília analysis

In [1]:
import pandas as pd
import numpy as np

from serenata_toolbox.datasets import fetch


fetch('2017-05-25-reimbursements.xz', '../data')
fetch('2017-05-21-companies-no-geolocation.xz', '../data') # we won't need geolocation for this analysis

In [2]:
reimbursements = pd.read_csv('../data/2017-05-25-reimbursements.xz',
                             dtype={'applicant_id': np.str,
                                    'cnpj_cpf': np.str,
                                    'congressperson_id': np.str,
                                    'subquota_number': np.str},
                             low_memory=False)

In [3]:
companies = pd.read_csv('../data/2017-05-21-companies-no-geolocation.xz', low_memory=False)
companies['cnpj'] = companies['cnpj'].str.replace(r'\D', '')

For this analysis Tony wanted the `Taxi, toll and parking` reimbursements from June 2014 to the present day. Let's filter that.

In [4]:
reimbursements = reimbursements[reimbursements['subquota_description'] == 'Taxi, toll and parking']
reimbursements2014 = reimbursements[reimbursements['year'] == 2014]
reimbursements2014 = reimbursements2014[reimbursements2014['month'] >= 6]
reimbursementsall = reimbursements[reimbursements['year'] >= 2015]
reimbursements = reimbursements2014.append(reimbursementsall)
reimbursements.head()

,year,applicant_id,document_id,reimbursement_value_total,total_net_value,reimbursement_numbers,congressperson_name,congressperson_id,congressperson_document,term,...,issue_date,document_value,remark_value,net_values,month,installment,passenger,leg_of_the_trip,batch_number,reimbursement_values
981025,2014,1001,5449172,0.0,371.68,4709,DILCEU SPERAFICO,73768,444.0,2015.0,...,2014-06-23 00:00:00,371.68,0.0,371.68,6,0,NaN,NaN,1123351,0.0
981074,2014,1001,5498708,0.0,458.08,4759,DILCEU SPERAFICO,73768,444.0,2015.0,...,2014-07-23 00:00:00,458.08,0.0,458.08,7,0,NaN,NaN,1136760,0.0
981075,2014,1001,5498713,0.0,731.48,4759,DILCEU SPERAFICO,73768,444.0,2015.0,...,2014-08-23 00:00:00,731.48,0.0,731.48,8,0,NaN,NaN,1136768,0.0
981134,2014,1001,5527835,0.0,628.68,4805,DILCEU SPERAFICO,73768,444.0,2015.0,...,2014-10-23 00:00:00,628.68,0.0,628.68,10,0,NaN,NaN,1144671,0.0
981822,2014,1003,5501016,0.0,15.00,4760,DOMINGOS DUTRA,74197,72.0,2011.0,...,2014-07-07 00:00:00,15.00,0.0,15.00,7,0,NaN,NaN,1137401,0.0


In [5]:
reimbursements.shape

(71237, 31)

We want to find the top 500 companies we have most expenses for

In [6]:
aggregation = reimbursements.groupby('cnpj_cpf')['total_net_value'].agg(np.sum).rename('sum').reset_index()
top500 = aggregation.sort_values('sum', ascending=False).head(500)
top500 = top500.reset_index(drop=True)
top500.head()

,cnpj_cpf,sum
0,04088208000165,564737.65
1,00031708000100,507986.50
2,07424109000103,57065.12
3,37990298000134,41974.85
4,60537263089981,41769.60


In [7]:
dataset = pd.merge(top500, companies, how='left', left_on='cnpj_cpf', right_on='cnpj')
dataset = dataset.drop('cnpj', axis=1)
dataset.head()

,cnpj_cpf,sum,additional_address_details,address,capital_social,city,email,extra,last_updated,legal_entity,...,situation,situation_date,situation_reason,special_situation,special_situation_date,state,status,trade_name,type,zip_code
0,04088208000165,564737.65,NaN,R MINAS BOGASIAN,99863241.68,OSASCO,NaN,{},2017-02-01T10:16:54.823Z,206-2 - Sociedade Empresária Limitada,...,ATIVA,09/10/2000,NaN,NaN,NaN,SP,OK,NaN,MATRIZ,06.013-010
1,00031708000100,507986.50,"BL. A SALAS 1,2 E 3",SHC/SUL SQ PLL PARTE,0.00,BRASILIA,NaN,{},2017-03-23T00:24:04.647Z,313-1 - Entidade Sindical,...,ATIVA,03/11/2005,NaN,NaN,NaN,DF,OK,SINPETAXI,MATRIZ,70.338-080
2,07424109000103,57065.12,SALA 16 SLJ,Q SCS QUADRA 05 BLOCO C,0.00,BRASILIA,NaN,{},2017-03-07T06:16:25.841Z,313-1 - Entidade Sindical,...,ATIVA,01/06/2005,NaN,NaN,NaN,DF,OK,SINTAXI,MATRIZ,70.305-000
3,37990298000134,41974.85,NaN,Q SCS QD. 01 BLOCO K SALA 1202,0.00,BRASILIA,NaN,{},2017-03-25T09:40:57.773Z,206-2 - Sociedade Empresária Limitada,...,ATIVA,03/11/2005,NaN,NaN,NaN,DF,OK,RADIO TAXI ALVORADA,MATRIZ,70.398-900
4,60537263089981,41769.60,NaN,AER INTERNACIONAL DE BRASILIA - P. JUSCELINO K...,0.00,BRASILIA,NaN,{},2017-02-19T14:10:19.999Z,205-4 - Sociedade Anônima Fechada,...,ATIVA,28/04/2014,NaN,NaN,NaN,DF,OK,ALLPARK EMPREENDIMENTOS PARTICIPACOES E SERVICOS,FILIAL,71.608-900


In [8]:
dataset.shape

(500, 405)

## Further investigation

With that table we selected a few companies and apps for further investigation:

1. Companies
   1. CGMP CENTRO DE GESTAO DE MEIOS DE PAGAMENTO LTDA - CNPJ: 04088208000165
   1. SINDICATO DOS PERMISSIONARIOS DE TAXIS E MOTORISTAS AUXILIARES DO DISTRITO FEDERAL - CNPJ: 00031708000100
   1. SINDICATO DOS TAXISTAS DO DISTRITO FEDERAL- SINTAXI - CNPJ: 7424109000103
   1. RADIO TAXI ALVORADA LTDA - ME - CNPJ: 37990298000134
   1. ALLPARK EMPREENDIMENTOS, PARTICIPACOES E SERVICOS S.A. - CNPJ: 60537263089981 
1. APPS
   1. 99 TAXIS LLC - CNPJ: 18533324000150
   1. 99 TAXIS DESENVOLVIMENTO DE SOFTWARES LTDA. - CNPJ: 18033552000161
   1. EASY TAXI SERVICOS LTDA. - CNPJ: 16809351000188
   1. UBER DO BRASIL TECNOLOGIA LTDA. - CNPJ: 17895646000187
   
   
We want to know the top 5 lower house representatives with most expenses in those CNPJs and how many expenses where there.

In [9]:
cnpjs = ['04088208000165', '00031708000100', '7424109000103',  '37990298000134', '60537263089981',
         '18533324000150', '18033552000161', '16809351000188', '17895646000187']
reimbursements = reimbursements[reimbursements['cnpj_cpf'].isin(cnpjs)]
reimbursements.head()

,year,applicant_id,document_id,reimbursement_value_total,total_net_value,reimbursement_numbers,congressperson_name,congressperson_id,congressperson_document,term,...,issue_date,document_value,remark_value,net_values,month,installment,passenger,leg_of_the_trip,batch_number,reimbursement_values
981025,2014,1001,5449172,0.0,371.68,4709,DILCEU SPERAFICO,73768,444.0,2015.0,...,2014-06-23 00:00:00,371.68,0.0,371.68,6,0,NaN,NaN,1123351,0.0
981074,2014,1001,5498708,0.0,458.08,4759,DILCEU SPERAFICO,73768,444.0,2015.0,...,2014-07-23 00:00:00,458.08,0.0,458.08,7,0,NaN,NaN,1136760,0.0
981075,2014,1001,5498713,0.0,731.48,4759,DILCEU SPERAFICO,73768,444.0,2015.0,...,2014-08-23 00:00:00,731.48,0.0,731.48,8,0,NaN,NaN,1136768,0.0
981134,2014,1001,5527835,0.0,628.68,4805,DILCEU SPERAFICO,73768,444.0,2015.0,...,2014-10-23 00:00:00,628.68,0.0,628.68,10,0,NaN,NaN,1144671,0.0
981822,2014,1003,5501016,0.0,15.00,4760,DOMINGOS DUTRA,74197,72.0,2011.0,...,2014-07-07 00:00:00,15.00,0.0,15.00,7,0,NaN,NaN,1137401,0.0


In [10]:
keys = ['congressperson_name', 'cnpj_cpf']

reimbursements_agg = reimbursements.groupby(keys)['total_net_value'] \
                                   .agg([np.sum, len]).rename(columns={'len':'expenses'})
reimbursements_agg.head(20)

sum  expenses
congressperson_name  cnpj_cpf                         
ACELINO POPÓ         00031708000100   155.00       3.0
                     37990298000134    90.00       1.0
                     60537263089981    12.00       1.0
ADAIL CARNEIRO       00031708000100   206.00      10.0
                     16809351000188    35.00       1.0
                     60537263089981    14.00       1.0
ADELMO CARNEIRO LEÃO 00031708000100  8098.96     355.0
                     16809351000188    32.00       1.0
                     37990298000134   237.79      11.0
                     60537263089981    32.40       1.0
ADEMIR CAMILO        00031708000100   100.00       2.0
                     16809351000188    62.10       2.0
                     60537263089981   100.00       5.0
ADILTON SACHETTI     00031708000100   885.55      25.0
                     37990298000134    42.00       1.0
ADRIAN               00031708000100    60.00       1.0
ADÉRMIS MARINI       00031708000100   262.00      10.0
                     04088208000165  1215.02       3.0
                     17895646000187   378.49      15.0
AELTON FREITAS       00031708000100   540.00      20.0

In [11]:
reimbursements_agg = reimbursements_agg.reset_index()
reimbursements_agg.head()

,congressperson_name,cnpj_cpf,sum,expenses
0,ACELINO POPÓ,00031708000100,155.0,3.0
1,ACELINO POPÓ,37990298000134,90.0,1.0
2,ACELINO POPÓ,60537263089981,12.0,1.0
3,ADAIL CARNEIRO,00031708000100,206.0,10.0
4,ADAIL CARNEIRO,16809351000188,35.0,1.0


In [12]:
reimbursements_agg.shape

(768, 4)

In [13]:
companies = companies[['name', 'cnpj']]
companies.head(20)

,name,cnpj
0,IVO M DIAS - ME,04636932000186
1,POSTO DE SERVICO CAMBOATA LTDA,28232643000180
2,AUTO POSTO PRINCIPE III LTDA,18005070000106
3,"ACELE - TREINAMENTO, ASSESSORIA E CONSULTORIA ...",17793398000163
4,CAIRU AUTO CENTER EIRELI - EPP,32021479000139
5,LOJAS AMERICANAS S.A.,33014556037600
6,POSTO DE GASOLINA SAO SEBASTIAO DE BELFORD ROX...,36562874000180
7,RSO COMUNICACAO EIRELI - ME,22939818000107
8,SOCIEDADE RADIO VALE DO JAGUARIBE LTDA - ME,07623838000180
9,AROS COMERCIO DE ALIMENTOS LTDA. - EPP,13917439000125


In [14]:
dataset = pd.merge(reimbursements_agg, companies, right_on='cnpj', left_on='cnpj_cpf')
dataset = dataset.drop('cnpj', axis=1)
dataset.head(20)

,congressperson_name,cnpj_cpf,sum,expenses,name
0,ACELINO POPÓ,00031708000100,155.00,3.0,SINDICATO DOS PERMISSIONARIOS DE TAXIS E MOTOR...
1,ADAIL CARNEIRO,00031708000100,206.00,10.0,SINDICATO DOS PERMISSIONARIOS DE TAXIS E MOTOR...
2,ADELMO CARNEIRO LEÃO,00031708000100,8098.96,355.0,SINDICATO DOS PERMISSIONARIOS DE TAXIS E MOTOR...
3,ADEMIR CAMILO,00031708000100,100.00,2.0,SINDICATO DOS PERMISSIONARIOS DE TAXIS E MOTOR...
4,ADILTON SACHETTI,00031708000100,885.55,25.0,SINDICATO DOS PERMISSIONARIOS DE TAXIS E MOTOR...
5,ADRIAN,00031708000100,60.00,1.0,SINDICATO DOS PERMISSIONARIOS DE TAXIS E MOTOR...
6,ADÉRMIS MARINI,00031708000100,262.00,10.0,SINDICATO DOS PERMISSIONARIOS DE TAXIS E MOTOR...
7,AELTON FREITAS,00031708000100,540.00,20.0,SINDICATO DOS PERMISSIONARIOS DE TAXIS E MOTOR...
8,AFONSO FLORENCE,00031708000100,2574.00,63.0,SINDICATO DOS PERMISSIONARIOS DE TAXIS E MOTOR...
9,AFONSO MOTTA,00031708000100,130.62,4.0,SINDICATO DOS PERMISSIONARIOS DE TAXIS E MOTOR...


In [15]:
dataset.shape

(768, 5)

In [16]:
dataset = dataset.sort_values(['sum', 'expenses'], ascending=[False, False]).reset_index(drop=True)
dataset.head()

,congressperson_name,cnpj_cpf,sum,expenses,name
0,ZECA DIRCEU,00031708000100,36916.73,1136.0,SINDICATO DOS PERMISSIONARIOS DE TAXIS E MOTOR...
1,MARCELO SQUASSONI,04088208000165,36553.56,24.0,CGMP CENTRO DE GESTAO DE MEIOS DE PAGAMENTO LTDA
2,ALESSANDRO MOLON,00031708000100,28513.10,691.0,SINDICATO DOS PERMISSIONARIOS DE TAXIS E MOTOR...
3,FRANCISCO FLORIANO,04088208000165,27596.54,31.0,CGMP CENTRO DE GESTAO DE MEIOS DE PAGAMENTO LTDA
4,VANDERLEI MACRIS,04088208000165,25612.84,17.0,CGMP CENTRO DE GESTAO DE MEIOS DE PAGAMENTO LTDA


In [17]:
# writer = pd.ExcelWriter('taxi-analysis.xlsx') # requires openpyxl
# dataset.to_excel(writer,'Sheet1')
# writer.save()